In [1]:
import pandas as pd
import requests
import torch
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split

: 

: 

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-german-cased", num_labels=2)
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")


2022-11-21 16:28:13.233600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model

: 

In [3]:

def fetch_rawtext_from_wiki(subject='Maschinelles Lernen') :
     
    url = 'https://de.wikipedia.org/w/api.php'
    params = {
                'action': 'parse',
                'page': subject,
                'format': 'json',
                'prop':'text',
                'redirects':''
            }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    raw_html = data['parse']['text']['*']
    soup = BeautifulSoup(raw_html,'html.parser')
    soup.find_all('p')
    text = ''
    
    for p in soup.find_all('p'):
        text += p.text
    return text

def preprocess_text(text):
    newtext=re.sub("\[.*?\]","_",text)
    i=0
    newtext=newtext.replace("._","_.")
    while i<1:
        newtext=newtext.replace("._",".")
        i+=1
    return newtext

def split_classify_text(text):
    sen_text=text.split('.')
    is_claim=[]
    for i in sen_text:
        if "_" in i:
            is_claim.append(True)
        else:
            is_claim.append(False)
    df=pd.DataFrame()
    df["sentence_raw"]=sen_text
    df["is_claim"]=is_claim


    return df

def split_train_test(df):
    train_df, test_df = train_test_split(df,test_size=.10)

    return train_df,test_df


# In[6]:




You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing BertForSequenceClassification: ['vocab_layer_norm.weight', 'distilbert.transformer.layer.3.attention.k_lin.weight', 'distilbert.transformer.layer.3.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.2.attention.q_lin.weight', 'distilbert.transformer.layer.3.attention.v_lin.bias', 'distilbert.transformer.layer.2.attention.out_lin.bias', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.5.ffn.lin1.weight', 'distilbert.transformer.layer.5.attention.out_lin.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.l

In [ ]:

# Run Classes
text=fetch_rawtext_from_wiki()
text_pre=preprocess_text(text)
df=split_classify_text(text_pre)
train_df,test_df = split_train_test(df)

In [6]:
model.train_model(train_df)

/Users/jannis/opt/anaconda3/envs/masterth-env/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/44 [00:00<?, ?it/s]

2022-11-21 16:02:27.705222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/6 [00:00<?, ?it/s]

: 

: 

In [ ]:
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

# model = AutoModelForMaskedLM.from_pretrained("bert-base-german-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

ImportError: 
AutoModelForMaskedLM requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForMaskedLM".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [ ]:
def tokenize_text(text):
    nlp = spacy.load("de_core_news_md")
    doc = nlp(sentences[0])
    print(doc.text)
    for token in doc:
        print(token.text, token.pos_, token.dep_)